In [4]:
##ossec-hids中的rule配置和测试数据可用于日志分析
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET
import xmltodict, json
import pandas as pd
import collections
import re
from pandas.io.json import json_normalize

pd.options.display.max_colwidth = 1000
filepath = r"F:\open-source-check\ossec-testing\tests"

In [5]:
def parseXMLToPd(file): 
    try:
        with open(file, "r", encoding='utf-8') as fd:
            #https://github.com/martinblech/xmltodict/issues/2
            data = xmltodict.parse('<root>{0}</root>'.format(fd.read()))['root']

            #get all the vars
            df_vars = pd.DataFrame()
            if "var" in data:
                try:
                    df_vars = pd.DataFrame(json_normalize(data["var"]))
                except Exception as e:
                    print(e)
                    raise e
            
            #replace all the vars in jsonStr
            jsonData = json.dumps(data["group"], indent=4)
            for index, row in df_vars.iterrows():
                key, value = row["@name"], row["#text"]
                #for Unrecognized escape sequence
                value = re.sub(r"([\\\/])", r"\\\1", value)
                jsonData = jsonData.replace('$'+key, value)
                
            #print(jsonData)  
            return pd.read_json(jsonData)
    except Exception as e:
        print(file)
        raise e
    
    return None
    
#print(parseXMLToPd(join(filepath, "web_appsec_rules.xml")).head())
#print(parseXMLToPd(join(filepath, "syslog_rules.xml")).head())
#print(parseXMLToPd(join(filepath, "mcafee_av_rules.xml")).head())

In [14]:
filelist = [f for f in listdir(filepath) if isfile(join(filepath, f))]
print(filelist)

regex=re.compile('^(log \d+ pass = )')
df = pd.DataFrame()
for file in filelist:
    datalist = []
    try:
        with open(join(filepath, file), "r", encoding="utf-8") as fd:
            for line in fd:
                if re.match(regex, line):
                    datalist.append(re.sub(regex, '', line))
        if len(datalist) > 0:
            df_tmp = pd.DataFrame(datalist, columns=["body"])
            df_tmp["group_name"] = re.sub(r"(\.ini)$", "", file)
            df = df.append(df_tmp, ignore_index = True)
    except Exception as e:
        print(e)

print(df.shape)
df.to_csv("ossec_testing.csv", sep=',', encoding='utf-8', index=False)     

['apache.ini', 'apparmor.ini', 'asterisk.ini', 'cimserver.ini', 'cisco_ios.ini', 'cpanel.ini', 'doas.ini', 'dovecot.ini', 'exim.ini', 'firewalld.ini', 'mailscanner.ini', 'modsecurity.ini', 'named.ini', 'netscreen.ini', 'nginx.ini', 'openbsd-dhcpd.ini', 'openbsd.ini', 'opensmtpd.ini', 'ossec.ini', 'pam.ini', 'postfix.ini', 'proftpd.ini', 'rsh.ini', 'samba.ini', 'sshd.ini', 'su.ini', 'sudo.ini', 'syslog.ini', 'sysmon.ini', 'systemd.ini', 'unbound.ini', 'vsftpd.ini', 'web_appsec.ini', 'web_rules.ini']
(188, 2)


In [15]:
print(df)